<p style="text-align: center;font-size: 40pt">Mapping node</p>

# Overview 

Objectives of this lesson:
- explain how a 3D mapping node in ROS works
- assist students in the construction of their first 3D map

# Introduction
In this lesson, we are going to look at [ethzasl_icp_mapper](https://github.com/ethz-asl/ethzasl_icp_mapping/tree/reintegrate/master_into_indigo_devel/ethzasl_icp_mapper), which is a 3D mapping node in ROS. This node is written in C++ and uses the Iterative Closest Point (ICP) algorithm in the background to register point clouds. In the first part of the lesson, we will look at the way this node works. Then, we are going to build a 3D map using this mapper.

# Basic pipeline
Although it might look complicated, in reality, it is quite easy to understand how a 3D mapper works. Here is an illustration of what happens inside:

<p style="text-align: center;">
<img src="images/simple_mapper.png" width="90%">
</p>

First, as input, the mapper takes a lidar point cloud and current [odometry](https://en.wikipedia.org/wiki/Odometry) information. The point cloud is used to build the map and to localize in it. The odometry information is used as a prior (i.e. an estimation) for localization. As output, the mapper produces a refined odometry as well as a map. The outputted map is used in the next iteration of mapping, where a new input point cloud and new odometry information are fed to the mapper.

Let's look more carefully at the *Registration* box inside the mapper. As you've seen in the [previous module](../registration/0-overview.ipynb), a registration algorithm takes two point clouds and finds the transformation that minimizes the alignment error between them. Here, the registration algorithm takes two point clouds and a transformation. This transformation is only used to roughly pre-align the point clouds, to make the registration job easier. Then, the registration algorithm just has to fine-tune this alignment. The two input point clouds are the latest point cloud produced by the lidar and the current map. The output of this step is a transformation representing the precise localization of the lidar within the map.

Now, let's have a look at the *Transformation* box. The only purpose of this step is to take the input point cloud and to align it to the map by applying the transformation computed previously. The output of this step is the transformed input point cloud.

The last box remaining is *Merge and maintenance*. This step takes the current map and the transformed input point cloud and merges them. This step also includes maintenance jobs, such as identifying sections of the map which are not relevant (e.g. moving objects). The output of this step is the new current map.

# Complete pipeline
Now that you understand the basics, let's add a two optional steps:

<p style="text-align: center;">
<img src="images/complicated_mapper.png" width="90%">
</p>

The *Input filters* box is a pre-processing step. It applies a series of <tt>data filters</tt>, which were introduced in the [processing lesson](../registration/2-lesson_processing.ipynb) of the previous module, to the input point cloud. A common use case for this step is to remove portions of the vehicle seen by the lidar from input clouds. If those portions are not removed, there will be a trail of points on the path of the vehicle in the map. The output of this step is the filtered input point cloud.

The *Map post filters* box is a post-processing step. It applies a series of <tt>data filters</tt> to the map produced at the end of an iteration. A common use case for this step is to remove the sections of the map that were marked as non-relevant in the *Merge and maintenance* step. The output of this step is the new current map.

# Configuration files
The ethzasl_icp_mapper node allows users to set many options to customize the mapping. With them, it is possible to set the [tf](../ros/4-lesson-ros-tf.ipynb) frame names, the sensor max range, the wanted resolution for the map, the probability of points being dynamic, etc. On the virtual machine provided for this lecture, all those options are set in the `~/percep3d_workspace/catkin_workspace/src/ethzasl_icp_mapping/ethzasl_icp_mapper/launch/husky/darpa_dynamic_mapper_ctu.launch` launch file. In those options, it is possible to provide three different configuration files. Those configuration files are used to control what is happening in the *Input filters*, *Registration* and *Map post filters* steps. On the virtual machine, those three configuration files are:
- `~/percep3d_workspace/catkin_workspace/src/ethzasl_icp_mapping/ethzasl_icp_mapper/launch/husky/darpa_input_filters_ctu.yaml`
- `~/percep3d_workspace/catkin_workspace/src/ethzasl_icp_mapping/ethzasl_icp_mapper/launch/husky/darpa_icp_param_ctu.yaml`
- `~/percep3d_workspace/catkin_workspace/src/ethzasl_icp_mapping/ethzasl_icp_mapper/launch/husky/darpa_mapPost_filters_ctu.yaml`

Here is an illustration of where those configuration files are used:

<p style="text-align: center;">
<img src="images/mapper.png" width="90%">
</p>

## Input filters
Here is the non-commented content of `darpa_input_filters_ctu.yaml`:
```
- BoundingBoxDataPointsFilter:
    xMin: -0.3
    xMax: 10
    yMin: -0.1
    yMax: 0.2
    zMin: -0.2
    zMax: 5
    removeInside: 1
 
- SurfaceNormalDataPointsFilter:
    knn: 12
    epsilon: 1.33
    keepNormals: 1
    keepDensities: 1
    
- MaxDensityDataPointsFilter:
    maxDensity: 1000.0
    
- ShadowDataPointsFilter:
    eps: 0.1
    
- SimpleSensorNoiseDataPointsFilter:
    sensorType: 0

- ObservationDirectionDataPointsFilter

- OrientNormalsDataPointsFilter:
    towardCenter: 1
```
The `BoundingBoxDataPointsFilter` is there to remove portions of the vehicle seen by the lidar. The `SurfaceNormalDataPointsFilter` is there to compute the density and normals associated to each point. The density information is needed because the point cloud is then filtered based on its density by the `MaxDensityDataPointsFilter`. As for the normals at each point, they must be computed since a Point-to-Plane error minimizer is used, as you will see in the next section. 

## ICP parameters
Here is the non-commented content of `darpa_icp_param_ctu.yaml`:
```
matcher:
  KDTreeMatcher:
    knn: 10
    maxDist: 1.5
    epsilon: 1

outlierFilters:
  - TrimmedDistOutlierFilter:
    ratio: 0.80
  - SurfaceNormalOutlierFilter:
    maxAngle: 0.42

errorMinimizer:
  PointToPlaneErrorMinimizer:

transformationCheckers:
  - DifferentialTransformationChecker:
      minDiffRotErr: 0.001
      minDiffTransErr: 0.01
      smoothLength: 2
  - CounterTransformationChecker:
      maxIterationCount: 100
  - BoundTransformationChecker:
      maxRotationNorm: 0.5
      maxTranslationNorm: 2
      
inspector:
  NullInspector

logger:
  FileLogger
```
At the top of the file, the kind of *matcher* to use is chosen. Matchers were introduced as association solvers in the [association lesson](../registration/2-lesson_association.ipynb) of the previous module. Here, a `KDTreeMatcher` matching with the 10 closest neighbors of each point is chosen. Then, just below, the kind of outlier filter is chosen. Outlier filters were covered in the [outliers lesson](../registration/3-lesson_outliers.ipynb) of the previous module. After that, the `PointToPlaneErrorMinimizer` is set as error minimizer. This kind of error minimizer minimizes the distance between a point in the reading cloud and the plane which is the closest in the reference cloud. Error minimizers were also covered in the previous module, in the [error minimizer lesson](../registration/4-lesson_error_minimization.ipynb). Finally, convergence conditions are stated in the `transformationCheckers`.

## Map post filters
Here is the non-commented content of `darpa_mapPost_filters_ctu.yaml`:
```
- SurfaceNormalDataPointsFilter:
    knn: 15

- CutAtDescriptorThresholdDataPointsFilter:
    descName: probabilityDynamic
    useLargerThan: 1
    threshold: 0.5
```
The first filter is used to re-compute the normals for each point of the map. Then, points which were marked as dynamic (i.e. with a probability of being dynamic higher than 50%) are removed from the map. Now that you understand how the mapper works, it is time to try it!

# Demonstration

## Downloading the data
In the terminal of your virtual machine, run the following commands:
```
cd ~/percep3d_data/
wget -O bag.zip https://ulavaldti-my.sharepoint.com/:u:/g/personal/vlkub_ulaval_ca/EfejYPI79DJDiiT4rG7Mc-gBXOzOcBjJDHLm7ETSTu_Qjg?download=1
unzip bag.zip
rm bag.zip
```
If everything goes as expected, you should end up with a file named `ugv_ctu_2020-02-26-19-44-51_alpha_course_percep3d.bag` in your `~/percep3d_data/` folder.

## Launching the mapping
In the terminal of your virtual machine, start a roscore:
```
roscore
```
Then, in another tab (press `CTRL+SHIFT+T` to open a new tab), run the following commands:
```
rosparam set use_sim_time true
roslaunch ethzasl_icp_mapper darpa_dynamic_mapper_ctu.launch
```
This will start the mapping node. However, nothing will happen because no data is published yet. In another tab, open rviz:
```
rviz
```
We will configure it to allow you to see better what is going on when the mapping starts. First, remove the grid display, we won't need it. Then click on the `add` button, choose the `By topic` tab and add the `Odometry` display under `/icp_odom`. Do the same thing again for the `PointCloud2` display under `/point_map`. Then, in the options of the `PointCloud2` display you just added, change the `Style` option to `Points`, the `Size (Pixels)` option to `2` and the `Alpha` option to `0.3`. Your rviz window should look similar to this:

<p style="text-align: center;">
<img src="images/rviz.png" width="90%">
</p>

Finally, in a new tab of the terminal, let the magic happen:
```
rosbag play ~/percep3d_data/ugv_ctu_2020-02-26-19-44-51_alpha_course_percep3d.bag --clock
```
NB If your computer is having a hard time, you can add the `--rate=0.5` to the command above to make the bagfile play at half of its rate.

It might take a few seconds before you see anything in rviz, but then you should see a map being built. The bagfile which is playing is a snippet from a DARPA competition. It lasts 50 minutes in total and contains point clouds of multiple floors that the robot explored. It is only after approximatively 2 minutes that the robot starts going down the stairs. You don't need to play the whole bagfile, just go explore the map and have some fun.

## Exporting the map
Once you are satisfied with the map, you can export it to a vtk file by running the following command in a new tab of the terminal:
```
rosservice call /dynamic_mapper/save_map "filename:
  data: '/home/student/Desktop/map.vtk'"
```

Once the map is done being saved, you can open it in Paraview and play with it.

# Conclusion
You should do the following activities to enhance your understanding of the concepts viewed in this lesson:
- modify the markdown by adding your own notes using `> my notes`.